<link rel="stylesheet" href="../../styles/theme_style.css">
<!--link rel="stylesheet" href="../../styles/header_style.css"-->
<link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/4.7.0/css/font-awesome.min.css">

<table width="100%">
    <tr>
        <td id="image_td" width="15%" class="header_image_color_6"><div id="image_img" class="header_image_6"></div></td>
        <td class="header_text"> ECG Analysis - Heart Rate Variability Parameters </td>
    </tr>
</table>

<div id="flex-container">
    <div id="diff_level" class="flex-item">
        <strong>Difficulty Level:</strong>   <span class="fa fa-star checked"></span>
                                <span class="fa fa-star checked"></span>
                                <span class="fa fa-star"></span>
                                <span class="fa fa-star"></span>
                                <span class="fa fa-star"></span>
    </div>
    <div id="tag" class="flex-item-tag">
        <span id="tag_list">
            <table id="tag_list_table">
                <tr>
                    <td class="shield_left">Tags</td>
                    <td class="shield_right" id="tags">extract&#9729;ecg&#9729;hrv</td> 
                </tr>
            </table>
        </span>
        <!-- [OR] Visit https://img.shields.io in order to create a tag badge-->
    </div>
</div>

Using an analogy with the programming paradigms, electrophysiological signals can be viewed as objects containing lots of information inside.
However obtatining knowledge from an object is only possible by accessing its attributes (characteristics).

In signal processing there is an identical logic, so, for extracting knowledge from signals (our objects), we need to identify their intrinsic characteristics (parameters).

The following description explains how to extract some parameters from ECG, commonly used for heart rate variability analysis (HRV).

<strong><span class="color2">List of HRV analysis parameters:</span></strong>
    <br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<strong><span class="color2">&#9740;</span></strong> Minimum, Maximum and Average RR Interval;
    <br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<strong><span class="color2">&#9740;</span></strong> Minimum, Maximum and Average Heart Rate (BPM);
    <br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<strong><span class="color2">&#9740;</span></strong> SDNN;
    <br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<strong><span class="color2">&#9740;</span></strong> rmsSD;
    <br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<strong><span class="color2">&#9740;</span></strong> NN20, pNN20;
    <br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<strong><span class="color2">&#9740;</span></strong> NN50, pNN50;
    <br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<strong><span class="color2">&#9740;</span></strong> Power inside ULF, VLF, LF and HF Frequency Bands;
    <br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<strong><span class="color2">&#9740;</span></strong> SD1, SD2, SD1 / SD2;

<hr>

<p class="steps">1 - Importation of the needed packages</p>

In [ ]:
# biosignalsnotebooks own package for loading and plotting the acquired data
import biosignalsnotebooks as bsnb

# Scientific packages
from numpy import linspace, max, min, average, std, array, diff, fabs, sqrt, power, round
from scipy.integrate import simps

<p class="steps">2 - Load of acquired ECG data</p>

In [ ]:
# Load of data
data, header = bsnb.load_signal("ecg_5_min", get_header=True)

<p class="steps">3 - Identification of the channel used during acquisition</p>

In [ ]:
channel = list(data.keys())[0]

In [ ]:
print (" Channel: " + str(channel) + " of " + str(header["device name"]) + " device")

<p class="steps">4 - Storage of sampling frequency and acquired data inside variables</p>

In [ ]:
# Sampling frequency and acquired data
fs = header["sampling rate"]

# Signal Samples
signal = data[channel]
time = linspace(0, len(signal) / fs, len(signal))

<p class="steps">5 - Generation of tachogram</p>
Tachogram defines the fundamental structure from where all parameters will be extracted.

In [ ]:
tachogram_data, tachogram_time = bsnb.tachogram(signal, fs, signal=True, out_seconds=True)

In [ ]:
bsnb.plot(tachogram_time, tachogram_data, x_axis_label='Time (s)', y_axis_label='Cardiac Cycle (s)', title="Tachogram",  x_range=(0, time[-1]))

<p class="steps">6 - Removal of ectopic beats</p>
A commonly accepted definition for ectopic beats establishes that a cardiac cycle that differs in at least 20 % of the duration of the previous one, can be considered an ectopic beat that should be removed.

In [ ]:
tachogram_data_NN, tachogram_time_NN = bsnb.remove_ectopy(tachogram_data, tachogram_time)
bpm_data = (1 / array(tachogram_data_NN)) * 60

<p class="steps">7 - Comparison between the tachograms obtained before and after ectopic beat removal</p>

In [ ]:
bsnb.plot_post_ecto_rem_tachogram(tachogram_time, tachogram_data, tachogram_time_NN, tachogram_data_NN)

<strong><span class="color7">We can conclude that there are not ectopic beats in the present acquisition</span></strong>

<p class="steps">8 - Extraction of Parameters</p>

<p class="steps">8.1 - Time Parameters</p>

In [ ]:
# Maximum, Minimum and Average RR Interval
max_rr = max(tachogram_data_NN)
min_rr = min(tachogram_data_NN)
avg_rr = average(tachogram_data_NN)

# Maximum, Minimum and Average Heart Rate
max_hr = 1 / min_rr # Cycles per second
max_bpm = max_hr * 60 # BPM

min_hr = 1 / max_rr # Cycles per second
min_bpm = min_hr * 60 # BPM

avg_hr = 1 / avg_rr # Cyles per second
avg_bpm = avg_hr * 60 # BPM

# SDNN
sdnn = std(tachogram_data_NN)

time_param_dict = {"Maximum RR": max_rr, "Minimum RR": min_rr, "Average RR": avg_rr, "Maximum BPM": max_bpm, "Minimum BPM": min_bpm, "Average BPM": avg_bpm, "SDNN": sdnn}

In [ ]:
print ("[Maximum RR, Minimum RR, Average RR] = [" + str(max_rr) + ", " + str(min_rr) + ", " + str(avg_rr) + "] s")
print ("[Maximum BPM, Minimum BPM, Average BPM] = [" + str(max_bpm) + ", " + str(min_bpm) + ", " + str(avg_bpm) + "] BPM")

In [ ]:
bsnb.plot_hrv_parameters(tachogram_time, tachogram_data, time_param_dict)

<p class="steps">8.2 - Poincar&#x00E9; Parameters</p>

In [ ]:
# Auxiliary Structures
tachogram_diff = diff(tachogram_data)
tachogram_diff_abs = fabs(tachogram_diff)
sdsd = std(tachogram_diff)
rr_i = tachogram_data[:-1]
rr_i_plus_1 = tachogram_data[1:]

# Poincaré Parameters
sd1 = sqrt(0.5 * power(sdsd, 2))
sd2 = sqrt(2 * power(sdnn, 2) - power(sd1, 2))
sd1_sd2 = sd1 / sd2

In [ ]:
print ("[SD1, SD2] = [" + str(sd1) + ", " + str(sd2) + "] s")
print ("SD1/SD2 = " + str(sd1_sd2))

In [ ]:
bsnb.plot_poincare(tachogram_data)

<p class="steps">8.3 - Frequency Parameters</p>

In [ ]:
# Auxiliary Structures
freqs, power_spect = bsnb.psd(tachogram_time, tachogram_data) # Power spectrum.

# Frequemcy Parameters
freq_bands = {"ulf_band": [0.00, 0.003], "vlf_band": [0.003, 0.04], "lf_band": [0.04, 0.15], "hf_band": [0.15, 0.40]}
power_values = {}
total_power = 0

band_keys = freq_bands.keys()
for band in band_keys:
    freq_band = freq_bands[band]
    freq_samples_inside_band = [freq for freq in freqs if freq >= freq_band[0] and freq <= freq_band[1]]
    power_samples_inside_band = [p for p, freq in zip(power_spect, freqs) if freq >= freq_band[0] and freq <= freq_band[1]]
    power = round(simps(power_samples_inside_band, freq_samples_inside_band), 5)
    
    # Storage of power inside each band
    power_values[band] = power
    
    # Total power update
    total_power = total_power + power

In [ ]:
print ("Power in [ULF, VLF, LF, HF] Bands = [" + str(power_values["ulf_band"]) + ", " + str(power_values["vlf_band"]) + ", " + str(power_values["lf_band"]) + ", " + str(power_values["hf_band"]) + "] s\u00B2")
print ("Total Power = " + str(total_power) + " s\u00B2")

In [ ]:
bsnb.plot_hrv_power_bands(freqs, power_spect)

<p class="steps">Additional Temporal Parameters</p>

In [ ]:
# Number of RR intervals that have a difference in duration, from the previous one, of at least 20 ms
nn20 = sum(1 for i in tachogram_diff_abs if i > 0.02)
pnn20 = int(float(nn20) / len(tachogram_diff_abs) * 100) # Percentage value.

# Number of RR intervals that have a difference in duration, from the previous one, of at least 50 ms
nn50 = sum(1 for i in tachogram_diff_abs if i > 0.05)
pnn50 = int(float(nn50) / len(tachogram_diff_abs) * 100) # Percentage value.

In [ ]:
print ("[NN20, pNN20, NN50, pNN50] = [" + str(nn20) + ", " + str(pnn20) + " %, " + str(nn50) + ", " + str(pnn50) + " %]")

*This procedure can be automatically done by **hrv_parameters** function in **extract** module of **<span class="color2">biosignalsnotebooks</span>** package*

In [ ]:
dictParameters = bsnb.hrv_parameters(signal, fs, signal=True)
print (dictParameters)

This set of parameters reveals interesting information about ECG signal, however you can extract much more features during your signal processing journey !

<strong><span class="color7">We hope that you have enjoyed this guide. </span><span class="color2">biosignalsnotebooks</span><span class="color4"> is an environment in continuous expansion, so don't stop your journey and learn more with the remaining <a href="../MainFiles/biosignalsnotebooks.ipynb">Notebooks <img src="../../images/icons/link.png" width="10px" height="10px" style="display:inline"></a></span></strong> ! 

<span class="color6">**Auxiliary Code Segment (should not be replicated by the user)**</span>

In [ ]:
bsnb.css_style_apply()

In [ ]:
%%html
<script>
    // AUTORUN ALL CELLS ON NOTEBOOK-LOAD!
    require(
        ['base/js/namespace', 'jquery'],
        function(jupyter, $) {
            $(jupyter.events).on("kernel_ready.Kernel", function () {
                console.log("Auto-running all cells-below...");
                jupyter.actions.call('jupyter-notebook:run-all-cells-below');
                jupyter.actions.call('jupyter-notebook:save-notebook');
            });
        }
    );
</script>